In [88]:
from __future__ import annotations

import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import re
from collections import Counter

pio.renderers.default = "notebook"
# %%

df = pd.read_csv("../data/output/finalized.csv", low_memory=False)

[c for c in df.columns if not c.endswith("_primary") and not c.endswith("_secondary")]

['OID_',
 'Join_Count',
 'TARGET_FID',
 'Join_Count_1',
 'TARGET_FID_1',
 'Join_Count_12',
 'TARGET_FID_12',
 'casenumber',
 'incident_date_x',
 'death_date_x',
 'age',
 'gender',
 'race',
 'latino',
 'cold_related',
 'heat_related',
 'commissioner_district',
 'residence_city',
 'residence_zip',
 'covid_related',
 'manner',
 'primarycause',
 'secondarycause',
 'gunrelated',
 'opioids',
 'primarycause_linea',
 'primarycause_lineb',
 'primarycause_linec',
 'geocoded_score',
 'geocoded_address',
 'full_address',
 'recovered',
 'final_latitude',
 'final_longitude',
 'closest_pharmacy',
 'CFNAME',
 'CFTYPE',
 'CFSUBTYPE',
 'ADDRESS',
 'GNISCODE',
 'SOURCE',
 'Jurisdicti',
 'Community',
 'STATEFP',
 'COUNTYFP',
 'GEOID',
 'INTPTLAT',
 'INTPTLON',
 'LANDUSE',
 'landuse_name',
 'landuse_sub_name',
 'landuse_major_name',
 'death_datetime',
 'death_time',
 'death_year',
 'death_month',
 'death_day',
 'death_week',
 'motel',
 'hot_combined',
 'cold_combined',
 'primary_combined',
 'repeated_addre

In [18]:
len([c for c in df.columns if c.endswith("_primary") or c.endswith("_secondary")])

134

In [19]:
[c for c in df.columns if not c.endswith("_primary") and not c.endswith("_secondary")]

['OID_',
 'Join_Count',
 'TARGET_FID',
 'Join_Count_1',
 'TARGET_FID_1',
 'Join_Count_12',
 'TARGET_FID_12',
 'casenumber',
 'incident_date_x',
 'death_date_x',
 'age',
 'gender',
 'race',
 'latino',
 'cold_related',
 'heat_related',
 'commissioner_district',
 'residence_city',
 'residence_zip',
 'covid_related',
 'manner',
 'primarycause',
 'secondarycause',
 'gunrelated',
 'opioids',
 'primarycause_linea',
 'primarycause_lineb',
 'primarycause_linec',
 'geocoded_score',
 'geocoded_address',
 'full_address',
 'recovered',
 'final_latitude',
 'final_longitude',
 'closest_pharmacy',
 'CFNAME',
 'CFTYPE',
 'CFSUBTYPE',
 'ADDRESS',
 'GNISCODE',
 'SOURCE',
 'Jurisdicti',
 'Community',
 'STATEFP',
 'COUNTYFP',
 'GEOID',
 'INTPTLAT',
 'INTPTLON',
 'LANDUSE',
 'landuse_name',
 'landuse_sub_name',
 'landuse_major_name',
 'death_datetime',
 'death_time',
 'death_year',
 'death_month',
 'death_day',
 'death_week',
 'motel',
 'hot_combined',
 'cold_combined',
 'primary_combined',
 'repeated_addre

In [3]:
df.closest_pharmacy.describe()

count    65227.000000
mean         1.304015
std         11.541382
min         -1.000000
25%          0.265323
50%          0.483335
75%          0.801440
max        572.601492
Name: closest_pharmacy, dtype: float64

In [4]:
drugs = pd.read_csv("../data/drugs/output.csv")
drugs.head()

,record_id,drug_name,word_found,similarity_ratio,tags
0,ME2022-03773,Covid,coronary,0.902273,covid
1,ME2022-03769,Covid,coronary,0.902273,covid
2,ME2022-03765,Covid,corona,0.909091,covid
3,ME2022-03725,Covid,coronary,0.902273,covid
4,ME2022-03716,Covid,coronary,0.902273,covid


In [5]:
drugs.similarity_ratio.describe()

count    6213.000000
mean        0.933585
std         0.037368
min         0.900000
25%         0.902273
50%         0.909091
75%         0.940000
max         1.000000
Name: similarity_ratio, dtype: float64

In [8]:
cases = pd.read_csv('../data/raw/cases.csv')
cases.shape

(65227, 33)

In [10]:
cases[(pd.isna(cases.latitude)) | (pd.isna(cases.longitude))].shape

(8528, 33)

In [11]:
df.recovered.sum()

7311

In [13]:
df.shape

(65227, 207)

In [16]:
cases[['latitude', 'longitude']].describe()

,latitude,longitude
count,56699.000000,56699.000000
mean,41.843895,-87.727874
std,0.139792,0.116211
min,41.469732,-88.263178
25%,41.750306,-87.778183
50%,41.858746,-87.707253
75%,41.941611,-87.649789
max,42.153747,-87.524835


In [17]:
df.geocoded_score.describe()

count    7311.000000
mean       93.855031
std         7.649486
min        70.000000
25%        89.830000
50%        97.300000
75%       100.000000
max       100.000000
Name: geocoded_score, dtype: float64

In [24]:
clean = pd.read_csv('../data/processed/recovered_lat_long.csv')
clean.columns

/Users/nick/Projects/Kentucky/geocoding/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning:

Columns (17,21) have mixed types.Specify dtype option on import or set low_memory=False.



Index(['casenumber', 'incident_date', 'death_date', 'age', 'gender', 'race',
       'latino', 'cold_related', 'heat_related', 'commissioner_district',
       'incident_street', 'incident_city', 'incident_zip', 'longitude',
       'latitude', 'location', 'residence_city', 'residence_zip', 'objectid',
       'chi_ward', 'chi_commarea', 'covid_related',
       ':@computed_region_nqe2_pztc', ':@computed_region_h3ai_7k6i',
       ':@computed_region_tu5p_2ban', 'manner', 'primarycause',
       'secondarycause', 'gunrelated', 'opioids', 'primarycause_linea',
       'primarycause_lineb', 'primarycause_linec', 'geocoded_latitude',
       'geocoded_longitude', 'geocoded_score', 'geocoded_address',
       'full_address', 'recovered', 'final_latitude', 'final_longitude'],
      dtype='object')

In [27]:
pd.notna(clean.full_address).sum()

7736

In [31]:
df.recovered.sum()

7311

In [33]:
pd.notna(clean.incident_street).sum()

64247

In [35]:
len(clean[pd.notna(clean.incident_street)].incident_street.unique())

61528

In [36]:
64247 - 61528

2719

In [38]:
len(df[pd.notna(df.full_address)].full_address.unique())

7087

In [39]:
pd.notna(df.full_address).sum()

7736

In [40]:
geo = pd.read_csv('../data/processed/recovered_lat_long.csv')
geo.head()

/Users/nick/Projects/Kentucky/geocoding/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning:

Columns (17,21) have mixed types.Specify dtype option on import or set low_memory=False.



,casenumber,incident_date,death_date,age,gender,race,latino,cold_related,heat_related,commissioner_district,...,primarycause_lineb,primarycause_linec,geocoded_latitude,geocoded_longitude,geocoded_score,geocoded_address,full_address,recovered,final_latitude,final_longitude
0,ME2022-03798,2022-03-06T22:00:00.000,2022-03-06T22:57:00.000,57.0,Female,Black,False,False,False,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,41.799746,-87.666927
1,ME2022-03797,2022-03-06T20:30:00.000,2022-03-06T20:37:00.000,37.0,Male,White,False,False,False,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,41.972986,-87.776603
2,ME2022-03796,2022-03-06T19:46:00.000,2022-03-06T19:59:00.000,36.0,Male,White,False,False,False,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,41.503817,-87.702314
3,ME2022-03795,2022-03-06T19:00:00.000,2022-03-06T19:06:00.000,21.0,Female,White,False,False,False,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,41.802975,-87.597731
4,ME2022-03794,2022-03-06T18:17:00.000,2022-03-06T18:53:00.000,33.0,Male,White,False,False,False,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,42.072090,-87.919917


In [86]:
cases.clean_address.value_counts()

1301 lee street                              17
339 9th ave                                  16
1920 nerge road                              14
345 dixie highway                            14
7001 w cullom ave                            14
                                             ..
165 n lavergne ave first floor                1
2317 south trumbull avenue                    1
parking lot of liquor store5510 s. racine     1
5825 w cermak rd room 723b                    1
3102 e. 91st street                           1
Name: clean_address, Length: 60029, dtype: int64

In [95]:
def city_sub(row: pd.Series) -> tuple[str, bool]:
    if pd.notna(row["incident_city"]):
        city = row["incident_city"].title().strip()
        subbed = False
    elif pd.isna(row["incident_city"]) and pd.notna(row["residence_city"]):
        city = row["residence_city"].title().strip()
        subbed = True
    else:
        city = ""
        subbed = False
    return city, subbed


def create_fixed_address(row: pd.Series) -> str | None:
    street = row["clean_address"] if row["clean_address"] else ""
    city = (
        row["incident_city"]
        if pd.notna(row["incident_city"]) and "unkn" not in row["incident_city"]
        else city_sub(row)[0]
    )
    zip_code = "" if pd.isna(row["incident_zip"]) else row["incident_zip"].strip()
    address = f"{street} {city} {zip_code}"
    if address == "":
        return None
    return address.strip()


def remove_apartment_info(x: str) -> str:
    # regex 1 to look for apartments and #s
    result1 = re.sub(r"apt.*|\#.*|.*nh,", "", x)
    # regex 2 to specify only alphanumeric + '.' for abbreviations and spaces
    result2 = re.sub(r"[^a-zA-Z0-9.\s]", "", result1)
    return result2


def clean_address(row: pd.Series) -> int | str | None:
    """Cleans an address by calling other utility functions.

    Args:
        row (pd.Series): row in a dataframe

    Returns:
        Union[int, str, None]: cleaned address or None
    """
    a = row["incident_street"]
    # handles 'unknown' and variations
    if pd.isna(a) or "unk" in a.lower() or "n/a" in a.lower():
        return None
    no_apartment_info = remove_apartment_info(a.lower())
    return no_apartment_info


In [100]:
cases['clean_address'] = cases.apply(clean_address, axis=1)
addrs = cases.apply(lambda row: create_fixed_address(row), axis=1)
len([x for x in cases.clean_address.value_counts().values if x > 1])

2083

In [84]:
s = set([a[0] for a in addrs])
cases.shape[0] - len(s)

4480

In [54]:
sum([a[1] for a in addrs])

1307

In [55]:
len([a[0] for a in addrs if a[0] is not None])

65227

In [58]:
all(a[0] for a in addrs)

True

In [59]:
pd.isna(cases.clean_address).sum()

1432

In [60]:
cases.clean_address.shape

(65227,)

In [61]:
65227 - 1432

63795

In [63]:
len(cases.clean_address.unique())

60030

In [66]:
pd.notna(cases.clean_address).sum() - len(cases[pd.notna(cases.clean_address)].clean_address.unique())

3766

In [73]:
df[['final_latitude', 'final_longitude']].describe()

,final_latitude,final_longitude
count,64010.000000,64010.000000
mean,41.839298,-87.739697
std,0.181251,0.160074
min,36.996540,-91.570947
25%,41.747631,-87.791285
50%,41.858579,-87.710390
75%,41.947060,-87.649965
max,42.569945,-87.380053


In [72]:
cases[['latitude', 'longitude']].describe()

,latitude,longitude
count,56699.000000,56699.000000
mean,41.843895,-87.727874
std,0.139792,0.116211
min,41.469732,-88.263178
25%,41.750306,-87.778183
50%,41.858746,-87.707253
75%,41.941611,-87.649789
max,42.153747,-87.524835


In [75]:
-87.739697 - -87.727874

-0.0118230000000068

In [103]:
df[df.closest_pharmacy > 0].closest_pharmacy.describe() * 0.62137

count    39734.747390
mean         0.838320
std          7.240096
min          0.000059
25%          0.172370
50%          0.306953
75%          0.503400
max        355.797389
Name: closest_pharmacy, dtype: float64

In [113]:
s1 = df[df.closest_pharmacy > 0]
s2 = s1[s1.opioids == 1]
miles = s2.closest_pharmacy * 0.62137
len([m for m in miles if m < 0.34]) / len(miles)

0.6132820841341021

In [78]:
drugs.head()

,record_id,drug_name,word_found,similarity_ratio,tags
0,ME2022-03773,Covid,coronary,0.902273,covid
1,ME2022-03769,Covid,coronary,0.902273,covid
2,ME2022-03765,Covid,corona,0.909091,covid
3,ME2022-03725,Covid,coronary,0.902273,covid
4,ME2022-03716,Covid,coronary,0.902273,covid


In [79]:
len(drugs.record_id.unique())

5787

In [80]:
cases.shape

(65227, 34)

In [81]:
drugs[drugs.drug_name == 'Covid'].shape

(3257, 5)

In [82]:
drugs[drugs.drug_name == 'Alcohol'].shape

(1890, 5)